In [420]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [421]:
os.chdir('/kaggle/input/stroke-prediction-dataset/')

In [422]:
!ls

# Import Data

In [423]:
df = pd.read_csv('healthcare-dataset-stroke-data.csv')

In [424]:
df

# Visualization Data

In [425]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.pairplot(df,hue='stroke')

In [426]:
sns.heatmap(df.corr(),annot=True)

# Data Cleaning

## Remove unnecessary columns 

In [427]:
df.drop('id',axis=1,inplace=True)
df.drop('bmi',axis=1,inplace=True)

In [428]:
df.describe()

In [429]:
sns.histplot(df['age'])

In [430]:
sns.catplot(x='stroke',y='age',hue='gender',data=df,kind='violin')


In [431]:
gender =  df['gender'].unique()
gender

In [432]:
gender_cnt = df['gender'].value_counts()
gender_cnt

In [433]:
sns.histplot(x=df['gender'].where(df['stroke']==1))

In [434]:
sns.histplot(x=df['gender'].where(df['stroke']==0))

### Check for Missing Data

In [435]:
print(df.isnull().sum().sort_values(ascending=False))

### Handle object Datatype

In [436]:
df.info()

In [437]:
df = pd.get_dummies(df,drop_first=True)

In [438]:
df

In [439]:
# Remove unnecessary columns 
df.drop('gender_Other',axis=1,inplace=True)
df.drop('work_type_Never_worked',axis=1,inplace=True)
df.drop('work_type_children',axis=1,inplace=True)
df.drop('smoking_status_never smoked',axis=1,inplace=True)
df.drop('smoking_status_smokes',axis=1,inplace=True)

In [440]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(15,15))
sns.heatmap(df.corr(),annot=True)
fig.show()

# Splitting Data

In [441]:
X = df.drop('stroke',axis=1)
y = df['stroke']

In [442]:
print(f' X_shape: {X.shape} \n y_shape: {y.shape}')

### Data Scaling

In [443]:
from sklearn.preprocessing import StandardScaler
scl = StandardScaler()
X = scl.fit_transform(X)

In [444]:
X.shape

### Split data into train and test

In [445]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,shuffle=True,random_state=0)
print(f' X_train: {X_train.shape} & X_test: {X_test.shape}')
print(f' y_train: {y_train.shape} & y_test: {y_test.shape}')

# Select the model

## Random Forest Classifier

In [446]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

y_pred_test = rf.predict(X_test)
y_pred_train = rf.predict(X_train)

In [447]:
from sklearn.metrics import accuracy_score
print('Accuracy score of Test: ', accuracy_score(y_test, y_pred_test)*100)
print('Accuracy score of Train: ', accuracy_score(y_train, y_pred_train)*100)

In [448]:
print(confusion_matrix(y_test, y_pred_test))

### Cross Validation

In [449]:
# Grid Search CV :Getting Best Hyper Parameters
rf.get_params().keys()

In [467]:
# from sklearn.model_selection import GridSearchCV

# parameters= {'n_estimators':[100,200],
#              'criterion':['gini','entropy','log_loss'],
#              'max_depth':[3,4,5,6,7,8,9],
#              'bootstrap' : [True, False]
#             }

# grid_search = GridSearchCV(estimator = rf,        # The Classifer That we need its best Parameters 
#                            param_grid = parameters,       # It must Be Dictionary or List Of Dictionaries 
#                            scoring = 'accuracy',          # The type of Evaluation Metric 
#                            cv = 5,                       # default None : Means K Fold =5 , you can change it to any 'int' Number
#                            n_jobs = -1)                 ## None :  For No Parallel Jobs , int : For a Certain Number of Parallel jobs , -1 : for Using ALL PROCESSOR)

# grid_search = grid_search.fit(X_train, y_train)
# print("best accuracy is :" , grid_search.best_score_ * 100)
# grid_search.best_params_ 

In [465]:
rf = rf = RandomForestClassifier(bootstrap=True,criterion='gini',max_depth=3,n_estimators=100)
rf.fit(X_train, y_train)

In [466]:
# Applying k-Fold Cross Validation
from sklearn.model_selection import cross_val_score
results = cross_val_score(estimator = rf, X = X_train, y = y_train, cv = 10)
print("Validation score: %.5f%% (%.5f%%)" % (results.mean()*100.0, results.std()*100.0))

y_pred = rf.predict(X_test)
rf_score = accuracy_score(y_test, y_pred)*100
print('Accuracy score: ', rf_score)

## evaluate the model

In [453]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, y_pred))

In [454]:
from sklearn.metrics import plot_confusion_matrix
disp = plot_confusion_matrix(rf, X_test, y_test,
                              display_labels=['no stroke','stroke'],
                              cmap=plt.cm.Blues)

In [455]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

### XGB Classifier

In [456]:
import xgboost as xgb
xgb_clf = xgb.XGBClassifier()

In [457]:
xgb_clf.get_params().keys()

In [458]:
parms = {
    'learning_rate' : [0.01,0.3,1,3],
    'n_estimators' : [900],
    'max_depth': [9,11]
}

grid = GridSearchCV(estimator = xgb_clf,        # The Classifer That we need its best Parameters 
                           param_grid = parms,       # It must Be Dictionary or List Of Dictionaries 
                           scoring = 'accuracy',          # The type of Evaluation Metric 
                           cv = 2,                       # default None : Means K Fold =5 , you can change it to any 'int' Number
                           verbose = 5
                   )                  

grid = grid.fit(X_train, y_train)
print("best accuracy is :" , grid.best_score_ * 100)
grid.best_params_ 

In [459]:
xgb_clf = grid.best_estimator_
xgb_clf.fit(X_train, y_train)

In [460]:
results = cross_val_score(estimator = xgb_clf, X = X_train, y = y_train, cv = 10)
print("Validation score: %.5f%% (%.5f%%)" % (results.mean()*100.0, results.std()*100.0))

y_pred = xgb_clf.predict(X_test)
xgb_score = accuracy_score(y_test, y_pred)*100
print('Accuracy score: ', xgb_score)

In [461]:
print(confusion_matrix(y_test, y_pred))

In [462]:
from sklearn.metrics import plot_confusion_matrix
disp = plot_confusion_matrix(xgb_clf, X_test, y_test,
                              display_labels=['no stroke','stroke'],
                              cmap=plt.cm.Blues)

In [463]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [464]:
scores = [rf_score,xgb_score]
labels = ['Random Forest Classifier', 'XGB Classifier']
accs = {'Accuracy': scores}
df = pd.DataFrame(accs,index=labels)
df